In [1]:
!pip install -q transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import pipeline

In [3]:
df = pd.read_csv('/content/text_classification_dataset_extended.csv')

In [4]:
label_encoder = LabelEncoder()

df['label'] = label_encoder.fit_transform(df['class'])

class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [5]:
df.head()

,text,class,label
0,Local team wins the championship!,sports,4
1,Stock market reaches all-time highs.,finance,0
2,Politician announces new policy.,political,3
3,City council approves new park.,local,2
4,International conference held in Paris.,international,1


In [6]:
df.shape

(40, 3)

In [7]:
dataset = Dataset.from_pandas(df[['text','label']])
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 40
})

In [8]:
print(dataset[0])

{'text': 'Local team wins the championship!', 'label': 4}


In [9]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [10]:
tokenized_text = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
label2id = {'finance': 0, 'international': 1, 'local': 2, 'political': 3, 'sports': 4}
id2label = {v: k for k, v in label2id.items()}

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    max_steps = 200,
    save_steps = 20,
    logging_steps = 5
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_text,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,1.618000
10,1.584900
15,1.466700
20,1.394100
25,1.313200
30,1.200900
35,1.107200
40,0.979700
45,0.828700
50,0.787400


TrainOutput(global_step=200, training_loss=0.46896500438451766, metrics={'train_runtime': 27.7194, 'train_samples_per_second': 28.861, 'train_steps_per_second': 7.215, 'total_flos': 2155815264360.0, 'train_loss': 0.46896500438451766, 'epoch': 20.0})

In [20]:
text = "bllomberg has stoped the stock rate news"

classifier = pipeline(task="text-classification", model="/content/my_model/checkpoint-80")
classifier(text)

[{'label': 'finance', 'score': 0.5053544044494629}]